## Import Modules & CSV to be Cleaned

In [1]:
#Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Import the CSV as a dataframe, called 'DF'
DF = pd.read_csv('C:\\ITStuff\\DataSets\\2016_Olympics.csv')

## Create and print function 'enhanced_info'

In [3]:
# Create and print an enhanced information table for our dataframe, called 'enhanced_info'
# First, we print the length of the DataFrame to provide a basic overview.
print(f"RangeIndex: {DF.shape[0]} entries, 0 to {DF.shape[0] - 1}")

# Next, we print the total number of columns in the DataFrame.
print(f"Total Columns: {DF.shape[1]}\n")

def enhanced_info(df):
    """
    Generate and print a summary table that provides an enhanced view of the DataFrame.
    
    The summary includes the name, data type, number of unique values, count of null values,
    and the percentage of empty cells for each column in the DataFrame.
    
    Parameters:
    - df (DataFrame): The DataFrame to analyze.

    The function also identifies and prints the total missing value proportion and a list of
    columns with missing values including their data types.
    """
    # Initialize a dictionary to store column information.
    info_data = {
        'Column Name': [],
        'Dtype': [],
        'UniqueValues': [],
        'NullCount': [],
        '%EmptyCells': [],
    }

    # A list to keep track of columns with missing values.
    columns_with_missing = []
    
    # Iterate over each column in the DataFrame to gather detailed information.
    for col in df.columns:
        # Count the number of null values in the column.
        null_count = df[col].isnull().sum()
        
        # Append the gathered information to the respective lists in the info_data dictionary.
        info_data['Column Name'].append(col)
        info_data['Dtype'].append(df[col].dtype)
        info_data['UniqueValues'].append(df[col].nunique())
        info_data['NullCount'].append(null_count)
        
        # Calculate the percentage of empty cells in the column.
        empty_cells_prop = (null_count / df.shape[0]) * 100
        info_data['%EmptyCells'].append(f'{empty_cells_prop:.2f}%')
        
        # If the column has missing values, add it to the columns_with_missing list.
        if null_count > 0:
            columns_with_missing.append((col, df[col].dtype))
    
    # Convert the info_data dictionary into a DataFrame for a tabular representation.
    info_df = pd.DataFrame(info_data)
    
    # Print the generated DataFrame.
    print(info_df.to_string(index=True))
    
    # Calculate and print the overall percentage of missing values in the DataFrame.
    overall_missing_proportion = df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100
    print(f'\nTotal missing data from the CSV: {overall_missing_proportion:.2f}%')
    
    # If there are columns with missing values, print them along with their data types.
    if columns_with_missing:
        print("\nColumns with Missing Values and Their Data Types:")
        for col, dtype in columns_with_missing:
            print(f"\t{col} - {dtype}")

# Call the 'enhanced_info' function with the DataFrame 'DF'.
enhanced_info(DF)

RangeIndex: 13688 entries, 0 to 13687
Total Columns: 15

   Column Name    Dtype  UniqueValues  NullCount %EmptyCells
0           ID    int64         11179          0       0.00%
1         Name   object         11174          0       0.00%
2          Sex   object             2          0       0.00%
3          Age    int64            48          0       0.00%
4       Height  float64            78        176       1.29%
5       Weight  float64           123        223       1.63%
6         Team   object           228          0       0.00%
7          NOC   object           207          0       0.00%
8        Games   object             1          0       0.00%
9         Year    int64             1          0       0.00%
10      Season   object             1          0       0.00%
11        City   object             1          0       0.00%
12       Sport   object            34          0       0.00%
13       Event   object           306          0       0.00%
14       Medal   object     

## Create function 'columns_with_missing'

In [4]:
# Function to identify columns with missing values and their data types, and returns them in a separate DataFrame
def columns_with_missing(df):
    """
    Identify and return a DataFrame containing columns with missing values and their data types.
    
    Parameters:
    - df (DataFrame): The DataFrame to analyze.
    
    Returns:
    - DataFrame: A DataFrame with columns 'Column Name' and 'Dtype' for columns with missing values.
    """
    missing_info = [(col, df[col].dtype) for col in df.columns if df[col].isnull().sum() > 0]
    columns_with_missing_df = pd.DataFrame(missing_info, columns=['Column Name', 'Dtype'])
    return columns_with_missing_df

## Create function 'total_missing'

In [5]:
# Function to calculate total missing data percentage
def total_missing(df):
    """
    Calculate and return the total percentage of missing data in the DataFrame.
    
    Parameters:
    - df (DataFrame): The DataFrame to analyze.
    
    Returns:
    - float: The total missing data percentage, rounded to two decimal places.
    """
    total_missing_data = df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100
    return round(total_missing_data, 2)

## Identify and Clean Columns with Mixed Data

In [6]:
# The below code reassigns the data type of each column into one of three types (string, numeric or other) if more than 95% of values are of that type.
# This function identifies the majority data type of each column
def check_column_data_types(dataframe):
    """
    Analyze each column in the DataFrame to determine its predominant data type 
    based on the content of the entries. A data type (string, numeric, or mixed) 
    is assigned to each column if more than 95% of its values belong to that type.

    Parameters:
    - dataframe (pd.DataFrame): The DataFrame whose columns are to be analyzed.

    Returns:
    - dict: A dictionary mapping each column name to its identified predominant 
            data type ('numeric', 'string', or 'mixed').
            
    Note:
    - Numeric data types are identified based on the ability to convert string 
      representations to floats or if the entry is already an integer or float.
    - The 'mixed' data type is assigned if no single data type accounts for more 
      than 95% of the entries in the column.
    """
    # Create an empty dictionary to hold the data types
    column_data_types = {}

    # Iterate over each column in the dataframe
    for column in dataframe.columns:
        # Initialize counters for different data types
        string_count = 0
        numeric_count = 0
        other_count = 0

        # Check the data type of each entry in the column
        for entry in dataframe[column]:
            if isinstance(entry, str):
                # Attempt to convert string to a float
                try:
                    float(entry)
                    numeric_count += 1
                except ValueError:
                    string_count += 1
            elif isinstance(entry, (int, float, np.number)):
                numeric_count += 1
            else:
                other_count += 1

        # Calculate the ratio of each data type
        total_entries = len(dataframe[column])
        numeric_ratio = numeric_count / total_entries
        string_ratio = string_count / total_entries

        # Set a threshold for determining the predominant data type
        threshold = 0.95

        # Assign the predominant data type based on the calculated ratios
        if numeric_ratio > threshold:
            column_data_type = 'numeric'
        elif string_ratio > threshold:
            column_data_type = 'string'
        else:
            column_data_type = 'mixed'

        column_data_types[column] = column_data_type

    return column_data_types

# Reassign column data types based on the identified majority data type, and change the values within those columns
def reassign_column_dtypes_with_na(dataframe, column_data_types):
    """
    Reassign the data types of DataFrame columns based on a given mapping of 
    columns to their predominant data types. Columns identified as 'numeric' 
    are converted to numeric types, with non-numeric entries set to NaN. Columns 
    identified as 'string' have non-string entries replaced with 'N/A'. Columns 
    marked as 'mixed' are left unchanged.

    Parameters:
    - dataframe (pd.DataFrame): The DataFrame whose columns are to be reassigned.
    - column_data_types (dict): A dictionary mapping column names to their 
                                predominant data type ('numeric', 'string', 
                                or 'mixed') as identified by check_column_data_types.

    Returns:
    - pd.DataFrame: The DataFrame with modified column data types according to 
                    the provided mapping.

    Note:
    - This operation modifies the input DataFrame in-place, potentially altering 
      data entries to conform to the identified predominant data type.
    """
    for column, dtype in column_data_types.items():
        if dtype == 'numeric':
            # Convert column to numeric, making non-numeric entries NaN
            dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce')
        elif dtype == 'string':
            # Convert all non-string entries to 'N/A'
            dataframe[column] = dataframe[column].apply(lambda x: x if isinstance(x, str) else 'N/A')
        elif dtype == 'mixed':
            # Leave mixed type columns as is for now
            pass
    return dataframe

# Analyze the data types for each column in the DataFrame
column_types = check_column_data_types(DF)

# Reassign the data types based on the predominant data type of each column
DF = reassign_column_dtypes_with_na(DF, column_types)

# You can now proceed with further data cleaning or analysis with the DataFrame
enhanced_info(DF)


   Column Name    Dtype  UniqueValues  NullCount %EmptyCells
0           ID    int64         11179          0       0.00%
1         Name   object         11174          0       0.00%
2          Sex   object             2          0       0.00%
3          Age    int64            48          0       0.00%
4       Height  float64            78        176       1.29%
5       Weight  float64           123        223       1.63%
6         Team   object           228          0       0.00%
7          NOC   object           207          0       0.00%
8        Games   object             1          0       0.00%
9         Year    int64             1          0       0.00%
10      Season   object             1          0       0.00%
11        City   object             1          0       0.00%
12       Sport   object            34          0       0.00%
13       Event   object           306          0       0.00%
14       Medal   object             3      11665      85.22%

Total missing data from

## Identify and reassign Categorical columns

In [7]:
# PID Categorical Columns
#Determine if a column should be treated as categorical. 
#If the number of unique values is less than 10% of the length then col is reassigned as 'categorical' dtype.
def is_categorical(column, threshold=0.1):
    """
    :column: pandas Series data (a column from our DataFrame)
    :threshold: float, the threshold ratio of unique values to total values for categorization
    :return: bool, True if <10%, False otherwise
    """
    # Calculate the ratio of unique values to the length of the column
    unique_ratio = column.nunique() / len(column)
    # If the ratio is less than the threshold, consider it as categorical
    return unique_ratio < threshold
    
#Reassign columns to 'category' where applicable.
def reassign_categorical_data_types(df):
    for col in df.select_dtypes(include=['object']).columns:  # Loop through 'object' type columns
        if is_categorical(df[col]):  # Check if the column is categorical
            df[col] = pd.Categorical(df[col])  # Convert to categorical type
    return df

# Apply the function to your DataFrame
DF = reassign_categorical_data_types(DF)

# Optionally, print the DataFrame's info to verify the changes
enhanced_info(DF)

   Column Name     Dtype  UniqueValues  NullCount %EmptyCells
0           ID     int64         11179          0       0.00%
1         Name    object         11174          0       0.00%
2          Sex  category             2          0       0.00%
3          Age     int64            48          0       0.00%
4       Height   float64            78        176       1.29%
5       Weight   float64           123        223       1.63%
6         Team  category           228          0       0.00%
7          NOC  category           207          0       0.00%
8        Games  category             1          0       0.00%
9         Year     int64             1          0       0.00%
10      Season  category             1          0       0.00%
11        City  category             1          0       0.00%
12       Sport  category            34          0       0.00%
13       Event  category           306          0       0.00%
14       Medal  category             3      11665      85.22%

Total m

# Imputing Missing Values

## Select columns by DataType

In [8]:
def get_user_choice():
    """
    Prompt the user to choose between numerical and categorical column handling.
    
    Returns:
    - choice: An integer representing the user's choice (1 for numerical, 2 for non-numerical).
    """
    print("Please select the type of columns you want to handle:")
    print("Press 1 for Numerical")
    print("Press 2 for Non-numerical")
    choice = input("Enter your choice (1/2): ")
    while choice not in ['1', '2']:
        print("Invalid input. Please enter 1 for Numerical or 2 for Non-numerical.")
        choice = input("Enter your choice (1/2): ")
    return int(choice)

def filter_columns_by_type(columns_with_missing, choice):
    """
    Filter the list of columns with missing values by the user's selected data type.
    
    Parameters:
    - columns_with_missing: A DataFrame containing the Column Name and Dtype of columns with missing values.
    - choice: An integer representing the user's choice (1 for numerical, 2 for non-numerical).
    
    Returns:
    - filtered_columns: A list of column names filtered by the selected data type.
    """
    if choice == 1:
        # Filter for numerical columns
        data_types = ['int64', 'float64']  # Common numerical data types in pandas
    else:
        # Filter for non-numerical (categorical) columns
        data_types = ['object', 'category', 'bool', 'datetime64[ns]', 'timedelta[ns]']  # Common non-numerical data types in pandas
    
    filtered_columns = [col for col, dtype in columns_with_missing if dtype in data_types]
    return filtered_columns

# Step 1: Get the user's choice
user_choice = get_user_choice()

# Step 2: Filter columns based on the user's choice
filtered_columns = filter_columns_by_type(columns_with_missing, user_choice)

# Output the filtered columns to verify
print("Selected columns for imputation:")
for col in filtered_columns:
    print(col)


Please select the type of columns you want to handle:
Press 1 for Numerical
Press 2 for Non-numerical


Enter your choice (1/2):  1


TypeError: 'function' object is not iterable